In [ ]:
import csv
import pandas as pd
import time
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)

def get_data():
    #this accesses a website that supposedly updates the vaccination data daily
    df=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")
    #this part gets the current date and converts it into the format of the csv file
    result = time.localtime(time.time())
    today=str(result.tm_year)+"-"+str(result.tm_mon)+"-"+str(result.tm_mday)
    #basically just filtered the data so we have the # of people fully vaccinated today, and daily # of vaccinations
    ph=df["location"].str.contains("Philippines")
    date=df[ph].iloc[-1]
    fully_vaxxed=date["people_fully_vaccinated"]
    vax_rate=date["daily_vaccinations"]
    total_vaccinations = date["total_vaccinations"]
    date_received = date["date"]
    date_received = datetime.strptime(date_received, '%Y-%m-%d')
    
    #number of newly infected + new deaths
    cases = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv")
    philippines = cases["location"].str.contains("Philippines")
    last_updated_date = cases[philippines]["last_updated_date"].values[0]
    new_case = float(cases[philippines]["new_cases"])
    new_death = float(cases[philippines]["new_deaths"])
    
    #webscrape population
    response = requests.get('https://worldpopulationreview.com/countries/philippines-population')
    page_text = response.text
    soup = BeautifulSoup(page_text)
    selector = '#__next > div > div > div.inside-body-content-container.clearfix > div:nth-child(1) > div.jsx-681953866.collapser.open > div > div > div.col-md-8 > div > div.center > span'
    view_population = soup.select(selector)
    population = view_population[0].text
    population = float(population.replace(',',''))
    
    #compute vaccine rate increase
    date_ytd = df[ph].iloc[-2]
    vax_rate_ytd  = date_ytd["daily_vaccinations"]
    increase = float(vax_rate-vax_rate_ytd)
    if increase > 0:
        percent_rate = f"+{int((increase/vax_rate_ytd)*100)}%"
    elif increase < 0:
        percent_rate  = f"-{int(-(increase/vax_rate_ytd)*100)}%"
    
    
    #compute herd immunity
    immunity_percentage = 0.7
    doses_for_immunity = population * immunity_percentage * 2
    days_until_herd_immunity = (doses_for_immunity - total_vaccinations)/vax_rate
    months_until_herd_immunity = round(days_until_herd_immunity / 30.417)
    herd_immunity_date = date_received + relativedelta(months=+months_until_herd_immunity)
    herd_immunity_date = herd_immunity_date.strftime('%B %Y')
    #return (vax_rate,fully_vaxxed, total_vaccinations, months_until_herd_immunity, herd_immunity_date,percent_rate)
    date_received = date_received.strftime("%Y-%m-%d")
    
    message = (f'''HERD IMMUNITY UPDATE (Philippines)
as of: {date_received}
Projected herd immunity date: {herd_immunity_date}
=======================================
COVID INFORMATION (Philippines)
Number of new cases: {new_case}
Number of new deaths: {new_death}
Number of fully vaccinated people: {fully_vaxxed}
Current daily vaccinations: {vax_rate}
Total vaccinations: {total_vaccinations}
Percent change of vaccinations: {percent_rate}
=======================================
Where to register for your free COVID-19 shot:
source updated on 14 June 2021
https://www.cnnphilippines.com/news/2021/3/29/LIST-COVID-19-vaccine-registration-LGUs.html''')
    return (message)

def vaccine(update: Update, context: CallbackContext):
    data=get_data()
    chat_id = update.message.chat_id
    context.bot.sendMessage(chat_id, data)

def main():
    updater = Updater('1902489502:AAEfXcLPcssx5uPpiCOme7gXBE3mgpqrjM8', use_context = True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('vaccine',vaccine))
    updater.start_polling()
    updater.idle()
    
if __name__ == '__main__':
    main()